# 4.6 - Redes Generativas Adversarias (GAN)

Además de los Autoencoders, otra de las arquitecturas más reconocidas para resolver problemas de aprendizaje no supervisado son las **Redes Generativas Adversarias (GAN)**.

Las GANs fueron introducidas por Ian Goodfellow en 2014 y consisten en dos modelos que se entrenan simultáneamente: un **generador** y un **discriminador**. 

- El **generador** intenta crear datos falsos que sean indistinguibles de los datos reales a partir de, típicamente, un vector de ruido Gaussiano.
- El **discriminador** intenta distinguir entre datos reales y falsos resolviendo un problema de clasificación binaria.

El proceso de entrenamiento se denomina *"adversario"* puesto que el generador intenta engañar al discriminador, y el discriminador trata de no ser engañado.

El objetivo del generador es maximizar la probabilidad de que el discriminador clasifique sus muestras generadas como reales. 

Por otro lado, el objetivo del discriminador es minimizar su tasa de error en la clasificación de las muestras reales y generadas.

<br>

En esta práctica crearemos una **Deep Convolutional Generative Adversarial Network (DCGAN)** para generar nuevas imágenes de dígitos del conjunto *MNIST*.

Ten en cuenta que, aunque sea su uso más común, no todas las GAN tienen por que ser convolucionales o generar imágenes, también pueden generar datos estructurados o tabulares.

## 4.6.1 - Conjunto de datos

Utilizaremos de nuevo el conjunto [MNIST](https://es.wikipedia.org/wiki/Base_de_datos_MNIST). Como recordarás, este conjunto posee imágenes en escala de grises con números del 0 al 9 escritos a mano por personas. Cada una de estas imágenes tiene una resolución de $28 \times 28$ píxels. Al ser en escala de grises, cada imagen será un tensor de $1 \times 28 \times 28$.

Este conjunto de datos está pensado para resolver un problema de *Aprendizaje supervisado de multiclasificación*, pero en este caso descartaremos las etiquetas y utilizaremos solamente las imágenes.

Para acelerar y simplificar el entrenamiento de la **DCGAN**, que puede ser temporalmente costoso, vamos a quedarnos solamente con imágenes de la clase 6.

De esta forma, nuestra red solo tendrá que aprender a generar imágenes de este dígito, y su tiempo de entrenamiento se reducirá considerablemente.

### Descargar conjunto

In [ ]:
import torch
from torchsummary import summary
from torch.utils.data import random_split
from torchvision import datasets, transforms

# Fijar la semilla para obtener reproducibilidad y crear variable device
seed = 42
torch.manual_seed(seed)  # Fijar semilla de PyTorch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Definimos una transformación que convierte las imágenes a tensores.
# La transformación ToTensor() convierte una imagen PIL en un tensor de PyTorch.
# El compose permite concatenar múltiples transformaciones, en este caso solo aplicamos una.
transform = transforms.Compose([transforms.ToTensor()])

# Descargamos el dataset indicando donde almacenarlo, la partición y las transformaciones
mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
# Para simplificar el problema, solo nos quedaremos con imágenes de la clase 6
selected_images = [idx for idx, l in enumerate(mnist_train) if l[1]==6]
mnist_train = torch.utils.data.Subset(mnist_train, selected_images)

batch_size = 64

# Tras descargar, ya tentemos un objeto Dataset, por lo que necesitamos un DataLoader
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=False)

## 4.6.2 - Arquitectura del modelo



### Red Generadora
Esta red se encargará de transformar un vector de ruido Gaussiano en $\mathbb{R}^{100}$ en una imagen de $1 \times 28 \times 28$. Para ello se aplicarán de forma consecutiva una serie de convoluciones transpuestas (deconvoluciones) que generarán el volumen deseado.

> **NOTA:** Hay que tener en cuenta que la red generadora ha de crear imágenes en el mismo rango de valores que las imágenes reales. De no hacerlo, el discriminador tendría muy facil su tarea de detectar imágenes verdaderas y falsas.
En este caso, las imágenes del conjunto MNIST ya vienen normalizadas en el rango $[0, 1]$, por tanto pondremos una función de activación *sigmoide* a la salida de nuestra red para obtener el mismo resultado.

In [ ]:
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn

class Generator(nn.Module):
   
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(100, 32*7*7)
        self.trans_conv1 = nn.ConvTranspose2d(32, 16, kernel_size = 3, stride = 2, padding = 1, output_padding = 1)
        self.trans_conv2 = nn.ConvTranspose2d(16, 8, kernel_size = 3, stride = 1, padding = 1)
        self.trans_conv3 = nn.ConvTranspose2d(8, 4, kernel_size = 3, stride = 1, padding = 1)
        self.trans_conv4 = nn.ConvTranspose2d(4, 1, kernel_size = 3, stride = 2, padding = 1, output_padding = 1)
    
    def forward(self, x):
        x = self.fc(x)
        x = x.view(-1, 32, 7, 7)
        x = F.relu(self.trans_conv1(x))
        x = F.relu(self.trans_conv2(x))
        x = F.relu(self.trans_conv3(x))
        x = self.trans_conv4(x)
        x = torch.sigmoid(x)
        
        return x        

G = Generator().to(device)
summary(G, (1, 100))

### Red Discriminadora
La segunda red es la encargada de clasificar, dada una imagen de $1 \times 28 \times 28$, en verdadera o generada mediante clasificación binaria. Para ello aplicaremos de forma consecutiva una serie de convoluciones que irán transformando un volumen de activaciones en otro hasta llegar a una única neurona de salida.

> **NOTA:** En este caso no se pone función de activación en la salida (sigmoide) puesto que se va a utilizar la loss *BCEWithLogitsLoss*, la cual incorpora esta función entre sus cálculos.



In [ ]:
class Discriminator(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv0 = nn.Conv2d(1, 16, kernel_size = 3, stride = 2, padding = 1)
        self.conv0_drop = nn.Dropout2d(0.25)
        self.conv1 = nn.Conv2d(16, 8, kernel_size = 3, stride = 1, padding = 1)
        self.conv1_drop = nn.Dropout2d(0.25)
        self.conv2 = nn.Conv2d(8, 4, kernel_size = 3, stride = 1, padding = 1)
        self.conv2_drop = nn.Dropout2d(0.25)
        self.adavg = nn.AdaptiveAvgPool2d((1, 1))
        self.flat = nn.Flatten()
        self.fc = nn.Linear(4, 1)
    
    def forward(self, x):
        x = F.leaky_relu(self.conv0(x), 0.2)
        x = self.conv0_drop(x)
        x = F.leaky_relu(self.conv1(x), 0.2)
        x = self.conv1_drop(x)
        x = F.leaky_relu(self.conv2(x), 0.2)
        x = self.conv2_drop(x)
        x = self.adavg(x)
        x = self.flat(x)
        x = self.fc(x)
        return x

D = Discriminator().to(device)
summary(D, (1, 28, 28))

A continuación tendremos que crear ambas redes y definir la loss que queremos para nuestro problema. Se han creado 4 funciones auxiliares que nos servirán a la hora de realizar el entrenamiento de la DCGAN:

1. ``discriminator_real_loss()``: Recibe las predicciones del discriminador $\mathbf{\hat{y}}$ para las imágenes reales y calcula la loss respecto de un vector de unos (la $\mathbf{y}$).
2. ``discriminator_fake_loss()``: Recibe las predicciones del discriminador $\mathbf{\hat{y}}$ para las imágenes generadas y calcula la loss respecto de un vector de ceros (la $\mathbf{y}$).
3. ``discriminator_loss()``: Acumula las losses anteriores, lo que resulta en la loss del discriminador.
4. ``generator_loss()``: Recibe las predicciones del discriminador ante las entradas del generador (generadas) y calcula la loss respecto de un vector de unos (la $\mathbf{y}$).

In [ ]:
Loss = nn.BCEWithLogitsLoss()

def discriminator_real_loss(real_out):
    real_label = torch.ones(real_out.size()[0], 1).to(device)
    real_loss = Loss(real_out.squeeze(), real_label.squeeze())
    return real_loss

def discriminator_fake_loss(fake_out):
    fake_label = torch.zeros(fake_out.size()[0], 1).to(device)
    fake_loss = Loss(fake_out.squeeze(), fake_label.squeeze())
    return fake_loss

def discriminator_loss(real_out, fake_out):
    real_loss = discriminator_real_loss(real_out)
    fake_loss = discriminator_fake_loss(fake_out)
    total_loss = (real_loss + fake_loss)
    return total_loss

def generator_loss(gen_disc_out):
    label = torch.ones(gen_disc_out.size()[0], 1).to(device)
    gen_loss = Loss(gen_disc_out.squeeze(), label.squeeze())
    return gen_loss

# Volvemos a crear los modelos para que, si cambiamos el LR, se haga reset.
G = Generator().to(device)
D = Discriminator().to(device)

# Para estabilizar el entrenamiento de una GAN, se recomienda utilizar beta1 = 0.5 en el Adam
disc_opt = optim.Adam(D.parameters(), lr = 0.0005, betas = (0.5, 0.999))
gen_opt =  optim.Adam(G.parameters(), lr = 0.0005, betas = (0.5, 0.999))

Finalmente ya solo quedaría la parte más compleja, entrenar el modelo. Para cada batch tendremos que:

1. *Entrenar el discriminador:*
    * Se obtienen las predicciones del discriminador para los ejemplos del bath (los reales).
    * Se generan tantos vectores de ruido en $\mathbb{R}^{100}$ como ejemplos tenga el batch y se pasan por el generador **(congelando previamente sus pesos)**.
    * Se obtiene la loss del discriminador a partir de las predicciones del mismo ante los ejemplos reales y los falsos.
    * Optimizamos el discriminador.

2. *Entrenar el generador:*
    * Se generan nuevos vectores de ruido y se pasan por el generador. En este caso generamos $2*batch\_size$ para que el generador y el discriminador se optimicen con el mismo número de ejemplos.
    * Se obtiene la predicción del discriminador ante los ejemplos anteriores y su loss correspondiente forzando al discriminador a predecir 1.
    * Optimizamos el generador.

In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import clear_output

def random_noise_generator(batch_size, dim):
    # Generar ruido normalizado entre -1 y 1
    return torch.rand(batch_size, dim)*2 - 1

def train(D, G, disc_opt, gen_opt, train_dl, batch_size, epochs = 25, gen_input_size = 100):
    
    # Listas para almacenar las losses
    disc_losses = []
    gen_losses = []
    
    # Para mostrar la evolución del generador cada época, generamos unos ejemplos aleatorios
    sample_size = 8
    fixed_samples = random_noise_generator(sample_size, gen_input_size)
    fixed_samples = fixed_samples.to(device)
    
    for epoch in range(epochs + 1):
        #Ponemos en modo Train ambos modelos
        D.train()
        G.train()
        
        disc_loss_total = 0
        gen_loss_total = 0
        gen_out = 0
        
        for train_x, _ in tqdm(train_dl):
            
            # -----------------------------------------------------------------------------
            # Entrenamiento del discriminador
            # -----------------------------------------------------------------------------
            
            # Reiniciar gradientes
            disc_opt.zero_grad() 
            
            # Generamos predicciones a partir de las imágenes reales
            train_x = train_x.to(device)
            real_out = D(train_x.float())     
            
            # Generamos tantos vectores de ruido como elementos hay en el batch (para alimentar el generador)
            disc_gen_in = random_noise_generator(batch_size, gen_input_size)
            disc_gen_in = disc_gen_in.to(device)
            
            # Obtenemos las predicciones del generador ante el ruido anterior.
            # Para congelar sus pesos, hay que hacer detach()
            disc_gen_out = G(disc_gen_in.float()).detach()
            fake_out = D(disc_gen_out.float())
            
            # Obtenemos la loss del discriminador, obtenemos gradientes y actualizamos pesos
            # Pretendemos que para las salidas reales prediga 1 y para las falsas 0
            disc_loss = discriminator_loss(real_out, fake_out)
            disc_loss_total += disc_loss.item()
            disc_loss.backward()
            disc_opt.step()  
            
            # -----------------------------------------------------------------------------
            # Entrenamiento del generador
            # -----------------------------------------------------------------------------

            # Reiniciar gradientes
            gen_opt.zero_grad()
            
            # Obtenemos las predicciones del generador para nuevo ruido.
            gen_gen_in = random_noise_generator(batch_size*2, gen_input_size)
            gen_gen_in = gen_gen_in.to(device)
            gen_out = G(gen_gen_in.float())     
            gen_disc_out = D(gen_out.float())       
            
            # Obtenemos la loss del discriminador, obtenemos gradientes y actualizamos pesos
            # Ahora pretendemos que para las salidas generadas prediga 1
            gen_loss = generator_loss(gen_disc_out) 
            gen_loss_total += gen_loss.item()
            gen_loss.backward()
            gen_opt.step()
        
        disc_losses.append(disc_loss_total)
        gen_losses.append(gen_loss_total)
        
        #Plotting samples
        G.eval()                    #Going into eval mode to get sample images         
        samples = G(fixed_samples.float())
        G.train()                   #Going back into train mode
        
        # Borrar la consola
        clear_output(wait=True)
        
        fig, axes = plt.subplots(figsize=(7,3), nrows=2, ncols=4, sharey=True, sharex=True)
        for ax, img in zip(axes.flatten(), samples):
            img = img.cpu().detach()
            ax.xaxis.set_visible(False)
            ax.yaxis.set_visible(False)
            im = ax.imshow(img.reshape((28,28)), cmap='Greys_r')
        plt.show()
        
        #Printing losses every epoch
        print("Epoch ", epoch, ": Discriminator Loss = ", disc_loss_total/len(train_dl), ", Generator Loss = ", gen_loss_total/len(train_dl))    
    
    return disc_losses, gen_losses

disc_losses, gen_losses = train(D, G, disc_opt, gen_opt, train_loader, epochs=100, batch_size=batch_size)

## 4.6.3. - Ejercicios

> **EJERCICIO:** Re-entrena y modifica la DCGAN anterior para que genere imágenes de los dígitos 0 y 1. Realiza un máximo de 50 épocas con un $ learning\_rate=0.001$.

> **EJERCICIO:** Crea y entrena una versión condicional de la red anterior que permita decidir que dígito generar. Para ello tendrás que incorporar una entrada adicional en el generador y discriminador indicando el tipo de dígito. 